In [59]:
import geopandas as gpd
import pandas as pd

In [60]:
# do spatial join
df_path_centerlines = gpd.read_file('')
df_neighbourhoods = gpd.read_file('')

In [61]:
# calculate the length of each centerline
df_path_centerlines['length'] = df_path_centerlines['geometry'].length 

In [62]:
# run spatial intersection to find for eah neighbourhood the paths which intersect it
df_join = gpd.sjoin(df_blocks, df_path_centerlines, op='intersects')

In [ ]:
# sumamrise stats

In [63]:
# get a summary per neighbourhood of the paths which intersect
data_ = {}

for i, row in df_join.iterrows():
    if row.fid not in data_.keys():
        data_[row.fid] = {'geometry':row.geometry, 'width':[], 'length':[]}
    data_[row.fid]['width'].append(row.width)
    data_[row.fid]['length'].append(df_path_centerlines.loc[row.index_right, 'length'])
    

In [64]:
# calculate the summary stats per neighbourhood
data = {'geometry':[], 'fid':[], 'width':[], 'no_of_paths':[], 'path_lengths':[]}

for fid in data_.keys():
    data['geometry'].append(data_[fid]['geometry'])
    data['fid'].append(fid)
    
    total_w = 0.0
    total_l = 0.0
    for j in range(0, len(data_[fid]['width'])):
        total_w += data_[fid]['width'][j] * data_[fid]['length'][j]
        total_l += data_[fid]['length'][j]
    
    data['width'].append(total_w/total_l)
    data['no_of_paths'].append(len(data_[fid]['width']))
    data['path_lengths'].append(total_l)
    

In [65]:
# create final dataframe
df2 = pd.DataFrame(data)
df2 = gpd.GeoDataFrame(df2, crs=df.crs, geometry='geometry')

In [67]:
# save final dataframe to file
df2.to_file('', driver='GPKG')